In [1]:
import numpy as np
import pandas as pd
from custom_transformers import add_combined_column, CombineAttributes, Square
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from warnings import filterwarnings

filterwarnings("ignore")

In [2]:
# Definimos um conjunto arbitrário para teste
numbers_array = np.array([1, 2, 3, 4, 5])
    
# Integramos nossa nova classe na API do scikit-learn
pipeline_quadrado = Pipeline([
    ("SquaredNumbers", Square())
])

pipeline_quadrado.fit_transform(numbers_array)

array([ 1,  4,  9, 16, 25])

In [3]:
pipeline_discreto = Pipeline([
    ("SumNumbers", FunctionTransformer(lambda x: x+10))
])

pipeline_discreto.fit_transform(numbers_array)

array([11, 12, 13, 14, 15])

In [4]:
data = pd.DataFrame({
    "Name": ["Adriano", "Ana", "Luis", "Guilherme", "José", "Maria", "Clara", "Leonardo", "Alice", "Júlia"],
    "Dorm": ["A", "B", "C", "A", "B", "C", "A", "B", "C", np.nan],
    "Notebooks": [2, 1, np.nan, 3, 2, 4, 1, 1, 2, 3]
})
data

,Name,Dorm,Notebooks
0,Adriano,A,2.0
1,Ana,B,1.0
2,Luis,C,NaN
3,Guilherme,A,3.0
4,José,B,2.0
5,Maria,C,4.0
6,Clara,A,1.0
7,Leonardo,B,1.0
8,Alice,C,2.0
9,Júlia,NaN,3.0


In [16]:
data_grouped = data.groupby("Dorm").sum(numeric_only=True)

rename_map = {"Notebooks": "Notebooks in Dorm"}
data_renamed = data_grouped.rename(columns=rename_map)

data_combined = data.join(data_renamed, on="Dorm")
data_combined

,Name,Dorm,Notebooks,Notebooks in Dorm
0,Adriano,A,2.0,6.0
1,Ana,B,1.0,4.0
2,Luis,C,NaN,6.0
3,Guilherme,A,3.0,6.0
4,José,B,2.0,4.0
5,Maria,C,4.0,6.0
6,Clara,A,1.0,6.0
7,Leonardo,B,1.0,4.0
8,Alice,C,2.0,6.0
9,Júlia,NaN,3.0,NaN


In [5]:
# Usando Pandas - Rápido e fácil. Perfeito para análise exploratória.
data_merged = add_combined_column(data, "Dorm", "Notebooks in Dorm")
data_merged

,Name,Dorm,Notebooks,Notebooks per Dorm
0,Adriano,A,2.0,6.0
1,Ana,B,1.0,4.0
2,Luis,C,NaN,6.0
3,Guilherme,A,3.0,6.0
4,José,B,2.0,4.0
5,Maria,C,4.0,6.0
6,Clara,A,1.0,6.0
7,Leonardo,B,1.0,4.0
8,Alice,C,2.0,6.0
9,Júlia,NaN,3.0,NaN


In [6]:
# Usando Scikit Learn - Possível de integrar ao Pipeline. Boa prática e melhor deploy.
Combiner = CombineAttributes(column="teste", to_integer=False)
Combiner.transform(data)

,Name,Dorm,Notebooks,Notebooks per Dorm
0,Adriano,A,2.0,6.0
1,Ana,B,1.0,4.0
2,Luis,C,NaN,6.0
3,Guilherme,A,3.0,6.0
4,José,B,2.0,4.0
5,Maria,C,4.0,6.0
6,Clara,A,1.0,6.0
7,Leonardo,B,1.0,4.0
8,Alice,C,2.0,6.0
9,Júlia,NaN,3.0,NaN


In [7]:
# Integrando um transformador personalizado ao Pipeline
main_pipe = Pipeline([
    
    ("Missing Inputing", ColumnTransformer([
        ("ImputerDorm", SimpleImputer(strategy="constant", fill_value="D"), ["Dorm"]),
        ("ImputerNotebook", SimpleImputer(strategy="constant", fill_value=0), ["Notebooks"])],
        remainder="drop",
        verbose_feature_names_out=False).set_output(transform="pandas")),

    ("Transformador personalizado", CombineAttributes()),

])

main_pipe

Pipeline(steps=[('Missing Inputing',
                 ColumnTransformer(transformers=[('ImputerDorm',
                                                  SimpleImputer(fill_value='D',
                                                                strategy='constant'),
                                                  ['Dorm']),
                                                 ('ImputerNotebook',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  ['Notebooks'])],
                                   verbose_feature_names_out=False)),
                ('Transformador personalizado', CombineAttributes())])

In [8]:
main_pipe.fit_transform(data)

,Dorm,Notebooks,Notebooks per Dorm
0,A,2.0,6
1,B,1.0,4
2,C,0.0,6
3,A,3.0,6
4,B,2.0,4
5,C,4.0,6
6,A,1.0,6
7,B,1.0,4
8,C,2.0,6
9,D,3.0,3
